In [1]:
import os
import sys

import time
import copy
import numpy as np
import math

sys.argv = ['options', '--classes', '40', '--dataroot', '/workspace/ModelNet/modelnet40-normal_numpy/',
           '--checkpoints_dir', '/workspace/ModelNet/SO-Net/SO-Net/checkpoints/']
from modelnet.options import Options
opt = Options().parse()  # set CUDA_VISIBLE_DEVICES before import torch

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np

from models.classifier import Model
from data.modelnet_shrec_loader import ModelNet_Shrec_Loader
from util.visualizer import Visualizer

from scipy import stats

------------ Options -------------
activation: relu
batch_size: 8
bn_momentum: 0.1
bn_momentum_decay: 0.6
bn_momentum_decay_step: None
checkpoints_dir: /workspace/ModelNet/SO-Net/SO-Net/checkpoints/
classes: 40
dataroot: /workspace/ModelNet/modelnet40-normal_numpy/
dataset: modelnet
device: cuda:0
display_id: 200
display_winsize: 256
dropout: 0.7
feature_num: 1024
gpu_id: 0
input_pc_num: 5000
k: 3
lr: 0.001
nThreads: 8
name: train
node_num: 64
normalization: batch
pretrain: None
pretrain_lr_ratio: 1
random_pc_dropout_lower_limit: 1
rot_horizontal: False
rot_perturbation: False
som_k: 9
som_k_type: avg
surface_normal: True
translation_perturbation: False
-------------- End ----------------


In [2]:
testset = ModelNet_Shrec_Loader(opt.dataroot, 'test', opt)
testloader = torch.utils.data.DataLoader(testset, batch_size=opt.batch_size, shuffle=False, num_workers=opt.nThreads)

In [3]:
model_probabilities_list = []
for model_name in os.listdir(opt.checkpoints_dir):
    print('Evaluating ' + model_name)
    if 'model' not in model_name:
        continue
    # create model, optionally load pre-trained model
    model = Model(opt)
    model_path = os.path.join(opt.checkpoints_dir, model_name)
    checkpoints = [x for x in os.listdir(model_path) if os.path.splitext(x)[-1] == '.pth']
    checkpoints = sorted(checkpoints, key=lambda x: int(x.split('_')[0]))[-2:]
    for item in checkpoints:
        if item.split('_')[-1] == 'classifier.pth':
            ckp_classifier = item
        if item.split('_')[-1] == 'encoder.pth':
            ckp_encoder = item
    classifier_ckp_path = os.path.join(model_path, ckp_classifier)
    encoder_ckp_path = os.path.join(model_path, ckp_encoder)
    model.encoder.load_state_dict(torch.load(encoder_ckp_path))
    model.classifier.load_state_dict(torch.load(classifier_ckp_path))
    model.test_loss.data.zero_()
    model.test_accuracy.data.zero_()
    model_probabilities = []
    gt_labels = []
    for i, data in enumerate(testloader):
        input_pc, input_sn, input_label, input_node, input_node_knn_I = data
        model.set_input(input_pc, input_sn, input_label, input_node, input_node_knn_I)
        model.test_model()
        model_probabilities.append(model.score.data)
        gt_labels.append(model.input_label.cpu())

    model_probabilities = torch.cat(model_probabilities, 0).cpu().numpy()
    gt_labels = torch.cat(gt_labels, 0).numpy()
    model_probabilities_list.append(model_probabilities)

Evaluating train
Evaluating model_7
Evaluating model_6
Evaluating model_2
Evaluating model_4
Evaluating model_10
Evaluating model_1
Evaluating model_5
Evaluating model_3
Evaluating model_8
Evaluating model_9


In [4]:
models_probabilities = np.stack(model_probabilities_list)
models_predictions = np.argmax(models_probabilities, -1)
correct_mask = np.equal(models_predictions, gt_labels).astype(np.float)
models_accuracy = np.mean(correct_mask, -1)
mean_accuracy = np.mean(models_accuracy)
max_accuracy = np.max(models_accuracy)
print('Mean model accuracy =', mean_accuracy)
print('Max model accuracy =', max_accuracy)

preditions_ens = np.squeeze(stats.mode(models_predictions)[0])
correct_mask_ens = np.equal(preditions_ens, gt_labels).astype(np.float)
accuracy_ens = np.mean(correct_mask_ens)
print('Ensembling (mode aggregation) accuracy =', accuracy_ens)

preditions_ens = np.argmax(np.mean(models_probabilities, 0), -1)
correct_mask_ens = np.equal(preditions_ens, gt_labels).astype(np.float)
accuracy_ens = np.mean(correct_mask_ens)
print('Ensembling (mean aggregation) accuracy =', accuracy_ens)

Mean model accuracy = 0.9264991896272285
Max model accuracy = 0.9294975688816856
Ensembling (mode aggregation) accuracy = 0.936385737439222
Ensembling (mean aggregation) accuracy = 0.936385737439222


In [6]:
models_probabilities_t = np.transpose(models_probabilities, axes=(1, 2, 0))
print(models_probabilities_t.shape, gt_labels.shape)

(2468, 40, 10) (2468,)


In [7]:
np.save('probabilities.npy', models_probabilities_t)
np.save('true_labels.npy', gt_labels)